In [1]:
import requests

CMR_SEARCH_URL = "https://cmr.earthdata.nasa.gov/search/"

def concept_ID_from_DOI(DOI: str) -> str:
    """
    Find the concept ID for a given DOI.

    Parameters:
    DOI (str): The DOI to search for.

    Returns:
    str: The concept ID for the given DOI.
    """
    URL = f"{CMR_SEARCH_URL}collections.json?doi={DOI}"
    response = requests.get(URL)

    if response.status_code != 200:
        raise ValueError(f"Error: {response.status_code} - {response.text}")

    concept_ID = response.json()['feed']['entry'][0]['id']

    return concept_ID


In [ ]:
DOI = "10.5067/VIIRS/VNP43IA4.002"
concept_ID_from_DOI(DOI)

In [ ]:
DOI = "10.5067/VIIRS/VNP21.002"
concept_ID_from_DOI(DOI)

In [ ]:
DOI = "10.5067/VIIRS/VNP21IMG_NRT.002"
concept_ID = concept_ID_from_DOI(DOI)
concept_ID

In [5]:
from typing import Union
from datetime import date, datetime
from dateutil import parser

def earliest_datetime(date_in: Union[date, str]) -> datetime:
    """
    Convert a date or date string to the earliest datetime of that date.

    Args:
        date_in (Union[date, str]): The input date or date string.

    Returns:
        datetime: The earliest datetime of the input date.
    """
    if isinstance(date_in, str):
        datetime_in = parser.parse(date_in)
    else:
        datetime_in = date_in

    date_string = datetime_in.strftime("%Y-%m-%d")
    return parser.parse(f"{date_string}T00:00:00Z")


def latest_datetime(date_in: Union[date, str]) -> datetime:
    """
    Convert a date or date string to the latest datetime of that date.

    Args:
        date_in (Union[date, str]): The input date or date string.

    Returns:
        datetime: The latest datetime of the input date.
    """
    if isinstance(date_in, str):
        datetime_in = parser.parse(date_in)
    else:
        datetime_in = date_in

    date_string = datetime_in.strftime("%Y-%m-%d")
    return parser.parse(f"{date_string}T23:59:59Z")


In [14]:
start_date_UTC = "2025-01-21"
end_date_UTC = "2025-01-21"

In [ ]:
import earthaccess

# Create the query with the concept ID and temporal range
query = earthaccess.granule_query() \
    .concept_id(concept_ID) \
    .temporal(earliest_datetime(start_date_UTC), latest_datetime(end_date_UTC))

query

In [16]:
from sentinel_tiles import sentinel_tiles

In [ ]:
geometry = sentinel_tiles.grid("11SPS")
geometry

In [18]:
from rasters import Point, Polygon, RasterGeometry

# Add spatial constraints to the query if provided
if isinstance(geometry, Point):
    # If the target geometry is a Point, add a point constraint to the query
    query = query.point(geometry.x, geometry.y)

if isinstance(geometry, Polygon):
    # If the target geometry is a Polygon, add a polygon constraint to the query
    ring = geometry.exterior
    
    # Ensure the ring is counter-clockwise
    if not ring.is_ccw:
        ring = ring.reverse()
    
    coordinates = ring.coords
    
    # Add the polygon coordinates to the query
    query = query.polygon(coordinates)

if isinstance(geometry, RasterGeometry):
    # If the target geometry is a RasterGeometry, add a polygon constraint to the query
    ring = geometry.corner_polygon_latlon.exterior
    
    # Ensure the ring is counter-clockwise
    if not ring.is_ccw:
        ring = ring.reverse()
    
    coordinates = ring.coords
    
    # Add the polygon coordinates to the query
    query = query.polygon(coordinates)

In [ ]:
query

In [20]:
from typing import List

class CMRServerUnreachable(Exception):
    """Exception raised when the CMR server is unreachable."""
    pass

# Execute the query and handle exceptions
granules: List[earthaccess.search.DataGranule]
try:
    granules = query.get()
except Exception as e:
    raise CMRServerUnreachable(e)

In [ ]:
granules